# Death on Two Legs, an Optimal Control Simulation
Addison Power, Jared Smith, Tyler Christensen, William Lewis

### Abstract
We attempt at modelling how to move a biped across a flat surface. Given a point of connection and 2 legs acting as double-pendulums, we try to minimize the distance between the robots final state and a given point (desired destination) using the angles of each leg as control.

## 1 Modeling
### 1.1 Initial Modeling
To model the robot's movement, we are treating the legs as double pendulums while in air but inverted double pendulums while on the ground. A $\theta$ parameter represents the angles of the "thigh" pendulums, and a $\phi$ parameter represents the angle of the "calf" pendulums. Our state $\vec{x}$, represents the position of the top of the legs where they connect (see figure below).

<b>TODO: add figure of robot</b>

Our model tries to minimize the amount the legs move, and the time spent moving for $\vec{x}$ to reach a final state. We constrain our control to possible angles similar to the standard human range of motion:

$$-10\degree\leq\theta_i \leq90\degree$$
$$-90\degree+\theta_i\leq\phi_i\leq0\degree+\theta_i$$

In the interest of generating a problem we can solve, we decide to abstract it from real world physics, seeing if we can encourage similar behavior mathematically. In other words, we do not consider things such as friction or mass but instead have rules such as the foot being on the ground should fix it in place or neither foot on the ground causes a negative velocity in the y direction.

Given the constraints in the previous section, we have devised the following cost functional, with our desired vertical position as $y^*$.

$$\int_0^{t_f}(\alpha_3 u_1)^2 + (\alpha_3 u_2)^2 + (\alpha_4 u_3)^2 + (\alpha_4 u_4)^2 + (\alpha(y - y^*)^4 dt$$

Since our control is defined on a compact set one might think the optimal control would end up being bang-bang but at any discrete timestep we want to avoid extreme motion in the legs. To achieve this we normalize the control by the desired maximum amount of motion at a timestep (such as 5 degrees of change from one frame to another) and then square this term leading to the $(\alpha_3 u_1)^2$ and similar terms in the cost function.

<b>TODO: add figure of cost graph </b>

We then call the following our state:

$$x_i' = l_2(\phi_i')\cos\phi_i + l1(\theta_i')\cos\theta_i$$
$$y_i' = l_2(\phi_i')\sin\phi_i + l_1(\theta_i')\sin\theta_i$$

This comes from finding the position of the top point of a single double pendulum and then taking the derivative with respect to time to try and find the force it exerts on the point $\vec{x}$. Also note the derivative terms, we handle these by adding them to our control, making it 8-dimensional. 

From here we then define the following condition:

\begin{equation*}c_i = 
\begin{cases}
\text{True} & y - (l_1\cos(\theta_i) + l_2\cos(\phi_i)) \leq 0 \\
\text{False} & \text{otherwise}
\end{cases}
\end{equation*}

This condition is supposed to tell use whether or not a foot (end of a double pendulum) is on the ground ($y=0$). Therefore, our state equation comes out to

\begin{align*}
\vec{x} &= [x', y']^T\\

x' &= \begin{cases}
x_1' + x_2' & c_1\text{ and }c_2 \\
x_1' & c_1 \text{ and not } c_2 \\
x_2' & c_2 \text{ and not } c_1 \\
0 & \text{otherwise}
\end{cases}\\

y' &= \begin{cases}
y_1' + y_2' & c_1\text{ and }c_2 \\
y_1' & c_1 \text{ and not } c_2 \\
y_2' & c_2 \text{ and not } c_1 \\
-\alpha_2 & \text{otherwise}
\end{cases}

\end{align*}

## 1.2 Deriving a System of ODEs